In [2]:
import pandas as pd
import numpy as np

data = pd.read_excel(r"D:\Thesis\2024\OE-NMC10Ah-45SOC_ssss.xlsx")

voltage_column = 'Voltage(V)'
temperature_columns = 'Temperature(°C)'
time_column_temp = 'Time(s)'
capacity_column = 'Capacity'
soc_column = 'SOC'

# Vinit (Initial Voltage)
Vinit = data[voltage_column].iloc[0]

# Vfinal (Final Voltage)
Vfinal = data[voltage_column].iloc[-1]

# Vrange (Voltage Range) = Max Voltage - Min Voltage
Vrange = data[voltage_column].max() - data[voltage_column].min()

# V2sec (Voltage at 2 seconds point)
V2sec = data[voltage_column].iloc[data[time_column_temp].sub(2).abs().idxmin()]

# V5sec (Voltage at 5 seconds point)
V5sec = data[voltage_column].iloc[data[time_column_temp].sub(5).abs().idxmin()]

# conditions to calculate Vscore as per paper
if Vrange / Vinit < 0.2:
    Vscore = 1
elif Vrange / Vinit > 0.5 and Vfinal / Vinit < 0.2:
    Vscore = 2
elif V2sec / Vinit < 0.4 and Vfinal / Vinit > 0.7:
    Vscore = 3
elif V2sec / Vinit >= 0.4 and Vfinal / Vinit > 0.7:
    Vscore = 4
elif V5sec / Vinit >= 0.95:
    Vscore = 5
else:
    Vscore = None 

data['Vscore'] = Vscore

# CTmax (maximum temperature)
max_temperature = data[temperature_columns].max().max()  # Find the maximum value across all temperature columns

#  T_rate (temperature rate)
temperature_rate = (
    data[temperature_columns].diff().abs().div(data[time_column_temp].diff(), axis=0).max().max()
)  

# nominal voltage
nominal_voltage = data[voltage_column].mean()  # Mean or median can be used

# wCap and wSOC
reference_capacity = 10000  
wCap = data[capacity_column] / reference_capacity
wSOC = data[soc_column] / 100  

# Weighted Coefficients
cScale = 95 / 6
cOffset = 5 - cScale
wA = 2.0 * cScale
wB = 3.0 * cScale
wC = 2.0 * cScale

# severity score
data['Scalc'] = (
    wA * (max_temperature / 160) ** 0.25 +
    wB * temperature_rate / 200 +
    wC * wCap * wSOC * data['Vscore'] +
    cOffset
)

data['Scalc'] = data['Scalc'].apply(lambda x: min(x, 100))

# severity level based on Scalc and Tmax
def determine_severity_level(row):
    Scalc = row['Scalc']
    if max_temperature < 40:
        return 1  # Very Low
    elif max_temperature < 80:
        if Scalc < 10:
            return 2  # Low
        elif Scalc < 25:
            return 3  # Moderate Low
        elif Scalc < 50:
            return 4  # Moderate
        elif Scalc < 75:
            return 5  # High
        elif Scalc < 90:
            return 6  # Very High
        else:
            return 7  # Extremely High
    else:
        return 7  # Extremely High

data['Severity_Level'] = data.apply(determine_severity_level, axis=1)

output_file_path = r"D:\Thesis\2024\severity.xlsx"  # Update the output path
data.to_excel(output_file_path, index=False)

print(f"Severity Score saved to: {output_file_path}") 


Severity Score saved to: D:\Thesis\2024\severity.xlsx
